## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

https://timm.fast.ai/


In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [29]:
from dotenv import load_dotenv
import wandb

load_dotenv()
api_key = os.getenv('WANDB_API_KEY')

wandb.login(key=api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


True

In [2]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [3]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [4]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        
        #print(f"------ preds's shape = {preds.shape},  targets's shape = {targets.shape}")
        #print(f"---------- preds = {preds},  targets = {targets}")
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }
    
    # wandb에 학습 과정 로그
    wandb.log(ret)

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [30]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 32
LR = 1e-3
EPOCHS = 1
BATCH_SIZE = 32
num_workers = 0

wandb.init(
    project="competition2-cv",
    config={
        "learning_rate": LR,
        "architecture": model_name,
        "dataset": "custom-dataset",
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "image_size": img_size
    }
)

In [6]:
device

device(type='cuda')

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [7]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [8]:
# Dataset 정의
trn_dataset = ImageDataset(
    "datasets_fin/train.csv",
    "datasets_fin/train/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "datasets_fin/sample_submission.csv",
    "datasets_fin/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


In [9]:
trn_dataset[0][1]

16

In [10]:
trn_dataset.df

array([['002f99746285dfdd.jpg', 16],
       ['008ccd231e1fea5d.jpg', 10],
       ['008f5911bfda7695.jpg', 10],
       ...,
       ['ff51dd281a8423f1.jpg', 11],
       ['ff8a6a251ce51c95.jpg', 5],
       ['ffc22136f958deb1.jpg', 9]], dtype=object)

In [11]:
tst_dataset.df

array([['0008fdb22ddce0ce.jpg', 0],
       ['00091bffdffd83de.jpg', 0],
       ['00396fbc1f6cc21d.jpg', 0],
       ...,
       ['ffc2c91dff8cf2c0.jpg', 0],
       ['ffc4e330a5353a2a.jpg', 0],
       ['ffc71fed753d90c1.jpg', 0]], dtype=object)

In [12]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,        # ??????
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [13]:
len(timm.list_models(pretrained=True))

1298

In [14]:
timm.list_models('efficientnet*', pretrained=True)

['efficientnet_b0.ra_in1k',
 'efficientnet_b1.ft_in1k',
 'efficientnet_b1_pruned.in1k',
 'efficientnet_b2.ra_in1k',
 'efficientnet_b2_pruned.in1k',
 'efficientnet_b3.ra2_in1k',
 'efficientnet_b3_pruned.in1k',
 'efficientnet_b4.ra2_in1k',
 'efficientnet_b5.sw_in12k',
 'efficientnet_b5.sw_in12k_ft_in1k',
 'efficientnet_el.ra_in1k',
 'efficientnet_el_pruned.in1k',
 'efficientnet_em.ra2_in1k',
 'efficientnet_es.ra_in1k',
 'efficientnet_es_pruned.in1k',
 'efficientnet_lite0.ra_in1k',
 'efficientnetv2_rw_m.agc_in1k',
 'efficientnetv2_rw_s.ra2_in1k',
 'efficientnetv2_rw_t.ra2_in1k']

In [15]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [16]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [17]:
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

  0%|          | 0/50 [00:00<?, ?it/s]

Loss: 2.5675: 100%|██████████| 50/50 [00:06<00:00,  7.61it/s]

train_loss: 2.4684
train_acc: 0.2720
train_f1: 0.2528
epoch: 0.0000



### TODO Validation

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [18]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:10<00:00,  9.17it/s]


In [19]:
preds_list

[12,
 15,
 16,
 3,
 8,
 2,
 9,
 12,
 5,
 16,
 5,
 4,
 16,
 9,
 2,
 2,
 2,
 5,
 3,
 12,
 12,
 6,
 4,
 10,
 3,
 2,
 12,
 5,
 6,
 16,
 12,
 3,
 2,
 2,
 16,
 2,
 15,
 3,
 7,
 10,
 5,
 8,
 16,
 2,
 0,
 8,
 5,
 9,
 11,
 0,
 7,
 12,
 2,
 16,
 7,
 3,
 9,
 5,
 4,
 4,
 2,
 12,
 5,
 9,
 2,
 6,
 5,
 16,
 10,
 9,
 12,
 7,
 5,
 16,
 5,
 13,
 7,
 6,
 6,
 0,
 9,
 9,
 16,
 16,
 2,
 9,
 5,
 12,
 12,
 0,
 6,
 16,
 16,
 10,
 12,
 16,
 5,
 2,
 9,
 2,
 9,
 12,
 16,
 6,
 6,
 12,
 8,
 2,
 2,
 5,
 7,
 16,
 16,
 2,
 10,
 16,
 2,
 6,
 2,
 5,
 16,
 6,
 16,
 8,
 2,
 16,
 6,
 15,
 15,
 9,
 16,
 3,
 12,
 8,
 16,
 16,
 4,
 15,
 2,
 9,
 12,
 9,
 6,
 2,
 2,
 16,
 9,
 16,
 9,
 9,
 12,
 8,
 15,
 3,
 11,
 2,
 9,
 9,
 6,
 8,
 16,
 8,
 5,
 3,
 16,
 12,
 9,
 6,
 5,
 9,
 2,
 16,
 9,
 9,
 9,
 9,
 3,
 16,
 4,
 4,
 10,
 2,
 5,
 9,
 2,
 2,
 2,
 3,
 9,
 9,
 2,
 5,
 2,
 5,
 12,
 15,
 9,
 12,
 0,
 5,
 6,
 16,
 16,
 2,
 2,
 6,
 5,
 0,
 0,
 10,
 5,
 2,
 6,
 5,
 15,
 16,
 0,
 3,
 0,
 12,
 2,
 4,
 16,
 0,
 6,
 9,
 12,
 0,
 15,
 2,
 5,
 

In [20]:
tst_dataset.df

array([['0008fdb22ddce0ce.jpg', 0],
       ['00091bffdffd83de.jpg', 0],
       ['00396fbc1f6cc21d.jpg', 0],
       ...,
       ['ffc2c91dff8cf2c0.jpg', 0],
       ['ffc4e330a5353a2a.jpg', 0],
       ['ffc71fed753d90c1.jpg', 0]], dtype=object)

In [21]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [22]:
pred_df

ID  target
0     0008fdb22ddce0ce.jpg      12
1     00091bffdffd83de.jpg      15
2     00396fbc1f6cc21d.jpg      16
3     00471f8038d9c4b6.jpg       3
4     00901f504008d884.jpg       8
...                    ...     ...
3135  ffb4b6f619fb60ea.jpg      15
3136  ffb54299b1ad4159.jpg       7
3137  ffc2c91dff8cf2c0.jpg       8
3138  ffc4e330a5353a2a.jpg       0
3139  ffc71fed753d90c1.jpg       0

[3140 rows x 2 columns]

In [23]:
sample_submission_df = pd.read_csv("datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [24]:
pred_df.to_csv("pred.csv", index=False)

In [25]:
pred_df.head()

ID  target
0  0008fdb22ddce0ce.jpg      12
1  00091bffdffd83de.jpg      15
2  00396fbc1f6cc21d.jpg      16
3  00471f8038d9c4b6.jpg       3
4  00901f504008d884.jpg       8

In [ ]:
# wandb 실행 종료
wandb.finish()